<a href="https://colab.research.google.com/github/vbloise3/Data_Science/blob/master/WorkingWithData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from typing import List , Dict 
from collections import Counter 
import math 
import matplotlib.pyplot as plt 
